In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier

In [4]:
df=pd.read_csv('./train.csv')

In [5]:
df

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,14549287,4.605170,Apartment,Private room,{},1,1.0,Real Bed,flexible,False,...,40.709025,-73.939405,one room bushwick,Williamsburg,0,NaN,https://a0.muscache.com/im/pictures/55162426/6...,11206.0,1.0,1.0
74107,13281809,5.043425,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,2.0,Real Bed,moderate,True,...,33.871549,-118.396053,Spacious Hermosa 2 BR on PCH,Hermosa Beach,16,93.0,https://a0.muscache.com/im/pictures/2b86560b-a...,90254,2.0,4.0
74108,18688039,5.220356,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",5,1.0,Real Bed,moderate,True,...,40.706749,-73.942377,Modern 2 Bedroom Apartment in Williamsburg,Williamsburg,43,94.0,https://a0.muscache.com/im/pictures/7fbe448c-5...,11206.0,2.0,2.0
74109,17045948,5.273000,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",2,1.0,Real Bed,strict,True,...,40.738535,-74.000157,Designer's Apartment in HEART of NYC,West Village,0,NaN,https://a0.muscache.com/im/pictures/b3971b63-0...,10011,0.0,2.0


In [6]:
# 실제로 train test split 할 특성
X=pd.DataFrame()

In [7]:
columns=['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'city',
       'latitude', 'longitude', 'neighbourhood', 'bedrooms', 'beds']

In [8]:
X=df[columns].copy()

In [9]:
X.shape

(74111, 14)

# GroupBy(City, Neighbourhood) 가격 평균

### null값 처리
- KNeighborsClassifier 사용

In [10]:
test_df=pd.DataFrame()
test_df=df[['longitude', 'latitude', 'neighbourhood']].copy()

test_df_not=test_df[test_df['neighbourhood'].notnull()]
test_df_null=test_df[test_df['neighbourhood'].isnull()]

In [11]:
X_t=test_df_not.drop(columns='neighbourhood')
y_t=test_df_not.iloc[:,-1]

In [12]:
label_enc=LabelEncoder()
label_enc.fit(y_t)
y_encoded=label_enc.transform(y_t)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y_encoded, test_size=0.2, random_state=42)

In [14]:
kne_clf = KNeighborsClassifier(n_neighbors=5)
kne_clf.fit(X_train, y_train)
pred=kne_clf.predict(X_test)

correct=(pred==y_test).sum()

correct/len(y_test)

C:\Users\YOOYOUNG\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.9561273051754908

In [15]:
test_df_null['neighbourhood']=kne_clf.predict(test_df_null.iloc[:,:2])

C:\Users\YOOYOUNG\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\YOOYOUNG\AppData\Local\Temp\ipykernel_13252\2726357634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_null['neighbourhood']=kne_clf.predict(test_df_null.iloc[:,:2])


In [16]:
test_df_null['neighbourhood']=label_enc.inverse_transform(test_df_null['neighbourhood'])

C:\Users\YOOYOUNG\AppData\Local\Temp\ipykernel_13252\2432593095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_null['neighbourhood']=label_enc.inverse_transform(test_df_null['neighbourhood'])


In [17]:
test_df_filled=pd.concat([test_df_not, test_df_null], axis=0)

In [18]:
X['neighbourhood']=test_df_filled['neighbourhood'].copy()

### 데이터 처리

In [19]:
test_df_filled['log_price']=df['log_price'].copy()

In [20]:
new = pd.DataFrame(df.groupby(['city', 'neighbourhood']).mean()['log_price'])

In [21]:
new_idx = new.reset_index()

In [22]:
all_df = pd.merge(left=new_idx, right=X, how='left',
                  on=['city','neighbourhood'], sort=False)

In [23]:
X=all_df.copy()

In [24]:
X.rename(columns={'log_price_x':'grouped_mean_price', 'log_price_y':'log_price'}, inplace=True)

# GroupBy(City, Neighbourhood) count

In [25]:
new = pd.DataFrame(df.groupby(['city', 'neighbourhood']).count()['id'])

In [26]:
new_idx = new.reset_index()

In [27]:
all_df = pd.merge(left=new_idx, right=X, how='left',
                  on=['city','neighbourhood'], sort=False)

In [28]:
X=all_df.copy()

In [29]:
X.rename(columns={'id':'grouped_count'}, inplace=True)

# X property_type...groupby해서 평균가격

In [30]:
property_mean_price=pd.DataFrame(df.groupby('property_type').mean()['log_price'])

In [31]:
new = pd.DataFrame(df.groupby('property_type').mean()['log_price'])

In [32]:
new_idx = property_mean_price.reset_index()

In [33]:
all_df = pd.merge(left=new_idx, right=X, how='left',
                  on=['property_type'], sort=False)

In [34]:
X=all_df.copy()

In [35]:
X.rename(columns={'log_price_x':'property_mean_price', 'log_price_y':'log_price'}, inplace=True)

# room_type, bed_type, cancel -> get_dummies

In [36]:
df['room_type'].nunique(), df['bed_type'].nunique(), df['cancellation_policy'].nunique()

(3, 5, 5)

In [37]:
X_merged=pd.concat([X, pd.get_dummies(X[['room_type', 'bed_type', 'cancellation_policy']])], axis=1)

In [38]:
X_merged.drop(columns=['room_type', 'bed_type', 'cancellation_policy'], inplace=True)

In [39]:
X=X_merged.copy()

# amenities 개수 추가하기

In [40]:
X['amenities']=list(map(lambda x:len(x.split(',')), X['amenities']))

# property_type, room_type....침대의 개수(평균개수)

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin

In [42]:
class bedsImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X['room_type']=df['room_type'].copy()
        self.avg_beds_by_room_type=X.groupby('room_type').mean()['beds']
        X.drop('room_type', inplace=True, axis=1)
        return self        
    
    def transform(self, X, y=None):
        X=X.copy()
        X['room_type']=df['room_type'].copy()
        X_null_beds = X[X['beds'].isnull()].index
        for idx in X_null_beds:
            X.at[idx, 'beds'] = self.avg_beds_by_room_type[X.at[idx, 'room_type']]
        X.drop('room_type', inplace=True, axis=1)
        return X
    
    def get_feature_names_out(self, feature_names_in):
        return feature_names_in

# property_type, room_type....침실의 개수(평균개수)

In [43]:
class bedroomsImputer(BaseEstimator, TransformerMixin):        
    def fit(self, X, y=None):
        
        X['room_type']=df['room_type'].copy()
        self.avg_bedrooms_by_room_type=X.groupby('room_type').mean()['bedrooms']
        X.drop('room_type', inplace=True, axis=1)
        return self
    
    def transform(self, X, y=None):
        X=X.copy()
        X['room_type']=df['room_type'].copy()
        X_null_bedrooms = X[X['bedrooms'].isnull()].index
        for idx in X_null_bedrooms:
            X.at[idx, 'bedrooms'] = self.avg_bedrooms_by_room_type[X.at[idx, 'room_type']]
        X.drop('room_type', inplace=True, axis=1)
        return X
    
    def get_feature_names_out(self, feature_names_in):
        return feature_names_in

# property_type, room_type....화장실의 개수(평균개수)

In [44]:
class bathroomsImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X['room_type']=df['room_type'].copy()
        self.avg_bathrooms_by_room_type=X.groupby('room_type').mean()['bathrooms']
        X.drop('room_type', inplace=True, axis=1)
        return self
    
    def transform(self, X, y=None):
        X=X.copy()
        X['room_type']=df.loc[:,'room_type']
        X_null_bathrooms = X[X['bathrooms'].isnull()].index
        for idx in X_null_bathrooms:
            X.at[idx, 'bathrooms'] = self.avg_bathrooms_by_room_type[X.at[idx, 'room_type']]
        X.drop('room_type', inplace=True, axis=1)        
        return X
    
    def get_feature_names_out(self, feature_names_in):
        return feature_names_in

In [45]:
class Dropcolumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X=X.copy()
        X.drop(columns=['longitude', 'latitude', 'property_type','city', 'neighbourhood'], inplace=True, axis=1)
        return X
    
#     def get_feature_names_out(self, feature_names_in):
#         return feature_names_in

In [80]:
X.columns

Index(['property_type', 'property_mean_price', 'city', 'neighbourhood',
       'grouped_count', 'grouped_mean_price', 'log_price', 'amenities',
       'accommodates', 'bathrooms', 'latitude', 'longitude', 'bedrooms',
       'beds', 'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room', 'bed_type_Airbed', 'bed_type_Couch',
       'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict', 'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60'],
      dtype='object')

In [46]:
y=X['log_price'].copy()
# X=X.drop(columns='log_price')

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [48]:
pre_pipeline = Pipeline([
    ('beds', bedsImputer()),
    ('bedrooms', bedroomsImputer()),
    ('bathrooms', bathroomsImputer()),
    ('dropcolumns', Dropcolumns()),
    ('std_scaler', StandardScaler())
    ])

In [49]:
X_prepared=pre_pipeline.fit_transform(X.drop('log_price', axis=1))

In [50]:
X_prepared

array([[-0.2453456 , -0.31320444, -1.0375202 , ..., -0.88071941,
        -0.03890416, -0.01514722],
       [-0.2453456 , -0.31320444, -1.0375202 , ...,  1.13543541,
        -0.03890416, -0.01514722],
       [-0.2453456 , -0.31320444, -1.0375202 , ..., -0.88071941,
        -0.03890416, -0.01514722],
       ...,
       [ 1.71485024, -0.64952798, -1.23212399, ...,  1.13543541,
        -0.03890416, -0.01514722],
       [ 1.71485024, -0.78232525, -0.39007849, ..., -0.88071941,
        -0.03890416, -0.01514722],
       [ 1.71485024, -0.72170041,  2.08013271, ..., -0.88071941,
        -0.03890416, -0.01514722]])

In [51]:
# X_prepared.drop(columns=['longitude', 'latitude', 'property_type','city', 'neighbourhood'], inplace=True)

# 모델 생성

In [52]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

In [53]:
X_prepared.shape, y.shape

((74111, 21), (74111,))

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size=0.2, random_state=42)

In [55]:
X_train.shape, X_test.shape

((59288, 21), (14823, 21))

In [56]:
y_train.shape, y_test.shape

((59288,), (14823,))

In [57]:
scores = {}

In [58]:
# LinearRegression
lin_reg = LinearRegression()
scores['lin_scores'] = cross_val_score(lin_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)

# RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=42)
scores['rf_scores'] = cross_val_score(rf_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)

# SGDRegressor
sgd_reg = SGDRegressor(penalty=None, eta0=0.001, random_state=42)
scores['sgd_scores'] = cross_val_score(sgd_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)

In [59]:
min_score = 1
best_model = 'best_model'

for k, v in scores.items():
    print('-{0:<14}/'.format(k), end=' ')
    score = np.sqrt(-v)
    print('/평균:', score.mean(), '/표준편차:', score.std())
    if min_score >= score.mean():
        min_score = score.mean()
        best_model = k
        
print('best_model:', best_model)

-lin_scores    / /평균: 0.4302918328975496 /표준편차: 0.006667393888454507
-rf_scores     / /평균: 0.43449473815057627 /표준편차: 0.006077491379839128
-sgd_scores    / /평균: 0.430345568147002 /표준편차: 0.0067070824045228775
best_model: lin_scores


In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
param_grid = {'n_estimators' : [10, 50, 100, 150, 200], 'max_features' : [4, 6, 8, 10]}

grid_search = GridSearchCV(rf_reg, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
%time grid_search.fit(X_train, y_train)

Wall time: 16min 47s


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_features': [4, 6, 8, 10],
                         'n_estimators': [10, 50, 100, 150, 200]},
             scoring='neg_mean_squared_error')

In [62]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 200}

In [63]:
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(np.sqrt(-mean_score), params)

0.44436536245005087 {'max_features': 4, 'n_estimators': 10}
0.4310976259780144 {'max_features': 4, 'n_estimators': 50}
0.4294229315513291 {'max_features': 4, 'n_estimators': 100}
0.42871278447361033 {'max_features': 4, 'n_estimators': 150}
0.42848948700211514 {'max_features': 4, 'n_estimators': 200}
0.4450825633116809 {'max_features': 6, 'n_estimators': 10}
0.4311961489191606 {'max_features': 6, 'n_estimators': 50}
0.4291794427321841 {'max_features': 6, 'n_estimators': 100}
0.4285490005943287 {'max_features': 6, 'n_estimators': 150}
0.4284740242700577 {'max_features': 6, 'n_estimators': 200}
0.4455651127072156 {'max_features': 8, 'n_estimators': 10}
0.43209848227583747 {'max_features': 8, 'n_estimators': 50}
0.4301188784349158 {'max_features': 8, 'n_estimators': 100}
0.4294511311686339 {'max_features': 8, 'n_estimators': 150}
0.4292965525230079 {'max_features': 8, 'n_estimators': 200}
0.44703315058597753 {'max_features': 10, 'n_estimators': 10}
0.43278159675444244 {'max_features': 10, 

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'n_estimators':randint(low=200, high=600), 'max_features':randint(low=1, high=8)}

random_search = RandomizedSearchCV(rf_reg, param_distribs, n_iter=10, scoring='neg_mean_squared_error', cv=5, n_jobs=-1) 
# n_iter : 반복 횟수

%time random_search.fit(X_train, y_train)

Wall time: 29min 51s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001E5D3C4A130>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001E5D3BDE970>},
                   scoring='neg_mean_squared_error')

In [65]:
random_search.best_params_

{'max_features': 5, 'n_estimators': 589}

In [66]:
cv_results = random_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(np.sqrt(-mean_score), params)

0.42789986598395563 {'max_features': 5, 'n_estimators': 589}
0.42801002139074357 {'max_features': 4, 'n_estimators': 376}
0.4308609446171341 {'max_features': 1, 'n_estimators': 217}
0.428111597390346 {'max_features': 6, 'n_estimators': 360}
0.427979936478499 {'max_features': 6, 'n_estimators': 465}
0.4287196495049835 {'max_features': 7, 'n_estimators': 269}
0.4297198282223856 {'max_features': 2, 'n_estimators': 209}
0.4293499387872352 {'max_features': 2, 'n_estimators': 400}
0.42928031806453715 {'max_features': 2, 'n_estimators': 476}
0.4304583500124867 {'max_features': 1, 'n_estimators': 338}


In [67]:
best_model = random_search.best_estimator_

**특성 중요도**

In [74]:
feature_importances = best_model.feature_importances_

In [83]:
total_attribs = ['property_type', 'property_mean_price', 'city', 'neighbourhood',
       'grouped_count', 'grouped_mean_price', 'log_price', 'amenities',
       'accommodates', 'bathrooms', 'latitude', 'longitude', 'bedrooms',
       'beds', 'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room', 'bed_type_Airbed', 'bed_type_Couch',
       'bed_type_Futon', 'bed_type_Pull-out Sofa', 'bed_type_Real Bed',
       'cancellation_policy_flexible', 'cancellation_policy_moderate',
       'cancellation_policy_strict', 'cancellation_policy_super_strict_30',
       'cancellation_policy_super_strict_60']
total_attribs

['property_type',
 'property_mean_price',
 'city',
 'neighbourhood',
 'grouped_count',
 'grouped_mean_price',
 'log_price',
 'amenities',
 'accommodates',
 'bathrooms',
 'latitude',
 'longitude',
 'bedrooms',
 'beds',
 'room_type_Entire home/apt',
 'room_type_Private room',
 'room_type_Shared room',
 'bed_type_Airbed',
 'bed_type_Couch',
 'bed_type_Futon',
 'bed_type_Pull-out Sofa',
 'bed_type_Real Bed',
 'cancellation_policy_flexible',
 'cancellation_policy_moderate',
 'cancellation_policy_strict',
 'cancellation_policy_super_strict_30',
 'cancellation_policy_super_strict_60']

In [85]:
feature_importances

array([0.0294061 , 0.07493497, 0.19672519, 0.09904141, 0.09890167,
       0.06480293, 0.10724655, 0.04356485, 0.14611648, 0.09554373,
       0.01865254, 0.00099167, 0.00073441, 0.00092778, 0.00087032,
       0.00223873, 0.0071688 , 0.005201  , 0.00646521, 0.00025005,
       0.0002156 ])

In [86]:
sorted(zip(feature_importances, total_attribs), reverse=True)

[(0.19672519222230483, 'city'),
 (0.14611648499550614, 'accommodates'),
 (0.10724654958517567, 'log_price'),
 (0.09904140943256771, 'neighbourhood'),
 (0.09890167201526842, 'grouped_count'),
 (0.0955437309485958, 'bathrooms'),
 (0.07493497012600131, 'property_mean_price'),
 (0.06480293317723515, 'grouped_mean_price'),
 (0.043564848779714206, 'amenities'),
 (0.029406098595225683, 'property_type'),
 (0.01865253836482439, 'latitude'),
 (0.007168804496052803, 'room_type_Shared room'),
 (0.0064652103622749495, 'bed_type_Couch'),
 (0.0052010006881799435, 'bed_type_Airbed'),
 (0.0022387271446313597, 'room_type_Private room'),
 (0.000991671161650003, 'longitude'),
 (0.0009277758144166667, 'beds'),
 (0.00087032214192032, 'room_type_Entire home/apt'),
 (0.0007344098725400101, 'bedrooms'),
 (0.0002500502647951715, 'bed_type_Futon'),
 (0.00021559981111945537, 'bed_type_Pull-out Sofa')]

# 모델 예측과 성능 평가

In [71]:
X_test.shape, y_test.shape

((14823, 21), (14823,))

In [72]:
from sklearn.metrics import mean_squared_error # MSE 구하기

final_predictions = best_model.predict(X_test)
final_rmse = mean_squared_error(y_test, final_predictions, squared=False) # squared=False : RMSE 구하기
final_rmse

0.42464173398059557

In [73]:
from scipy.stats import t # t분포

# 추정량(오차의 제곱들의 합)
squared_errors = (final_predictions - y_test)**2

# 95% 신뢰구간
confidence = 0.95

# 표본의 크기
n = len(squared_errors)

# 자유도(degree of freedom)
dof = n - 1 

# 추정량의 평균
m_squared_error = np.mean(squared_errors)

# 표본의 표준편차(비편형 분산으로 구함)
sample_std = np.std(squared_errors, ddof=1) # n-1로 나눈 표준편차

# 표준 오차
std_err = (sample_std/n)**0.5

# t.interval(신뢰구간, 자유도, 추정량의 평균(or MSE), 표준오차) # 신뢰구간 계산
mse_ci = t.interval(confidence, dof, m_squared_error, std_err)
rmse_ci = np.sqrt(mse_ci)
rmse_ci # RMSE에 대한 95% 신뢰 구간

array([0.41124427, 0.43762924])